# Assignment 1 

Database: (50_Startups)

You are being hired as a Data Scientist client wants
to create a model that can predict the profit of
the company based on company's location and Rd expense
(features : R&D Spend,State(company's location) )
Client expects the minimum accuracy of 0.92 of the deployed model

Your job is to check the feasibility of the project and create best training 
and testing splits


In [ ]:
# we here forced to just use 2 columns: R&D and location.

''' 
  but ideally we need to do all these steps to get good result:
  - EDA
  - Statistical Test
  - Feature Engineering
  - Then we do cross validation
  - 

'''
# if our base algorithms with the base parameters is giving us, we dont go for hyperparameter tuning.
# if we dont get result with base algorithms with the base parameters, we will go for hyperparameter tuning.

# We go to hyperparameter tuning, when the defalut parameters dont give us a desire output.

In [154]:
import numpy as np
import pandas as pd

In [155]:
data = pd.read_csv('50_Startups.csv')

In [156]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [163]:
#data[0]
data.iloc[0].values

array([165349.2, 136897.8, 471784.1, 'New York', 192261.83], dtype=object)

In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


# Data Preprocessing

In [158]:
data.isna().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [160]:
data['R&D Spend'].fillna(round(data['R&D Spend'].mean()), inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [11]:
#data['Marketing Spend'].fillna(round(data['Marketing Spend'].mean()), inplace=True)

In [161]:
data['State'].fillna(data['State'].mode()[0], inplace=True)

In [60]:
#features = data.iloc[:,:-1].values
#label = data.iloc[:,[-1]].values

In [75]:
#finalData = pd.concat([pd.get_dummies(data.State, dtype=int), data.iloc[:,[0,1,2,4]]] , axis = 1)
#finalData.head()

In [164]:
# Categorical Data Handling
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
fState = ohe.fit_transform(np.array(data['State']).reshape(-1,1))
#features = np.concatenate([fState, np.array(data.iloc[:, [0,1,2]])], axis=1)

In [66]:
#features = finalData.iloc[:,:-1].values
#label = finalData.iloc[:,[-1]].values

In [165]:
features = np.concatenate([fState, np.array(data.iloc[:, [0]])], axis=1)
label = data.iloc[:, [-1]].values


# KNeighborsRegressor

In [166]:
from sklearn.neighbors import KNeighborsRegressor
modelTest = KNeighborsRegressor()

In [167]:
from sklearn.model_selection import cross_val_score

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

scores = cross_val_score(modelTest,
                        features,
                        label,
                        cv = 10)

scores

array([ -9.52847973,  -2.42061691,  -7.28965118, -22.85356329,
        -0.37428142,  -8.95243978, -33.5713245 ,  -1.3902268 ,
        -1.76905992,  -3.65968395])

In [168]:
# What is the minimum score threshold for this dataset?

print("Minimum Score Threshold is : ",scores.mean())
print("Suggested SL value to commit: ", 1-scores.mean())

Minimum Score Threshold is :  -9.180932748418233
Suggested SL value to commit:  10.180932748418233


In [169]:
scores.max()

-0.37428142250826335

In [ ]:
#Goal: To identify the best Hyperparamter combinations for KNN w.r.t 50_Startups dataset
#      that can give me the score > 0.92

# Method 1 --- Using GridSearchCV

In [111]:
'''

KNeighborsRegressor(               
n_neighbors=5,	               Positive Integer
*,
weights='uniform',             ‘uniform’, 'distance'
algorithm='auto',              ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’
leaf_size=30,                  Positive Integer
p=2,                           1 (Manhattan distance),   2(Elucidean distance)
metric='minkowski',            "“euclidean”, “manhattan”, “chebyshev”, “minkowski”
                               “wminkowski”, “seuclidean”, “mahalanobis”"
metric_params=None,            Dictionary
n_jobs=None                    -1, 1 or -2
)

'''

'\n\nKNeighborsRegressor(               \nn_neighbors=5,\t               Positive Integer\n*,\nweights=\'uniform\',             ‘uniform’, \'distance\'\nalgorithm=\'auto\',              ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’\nleaf_size=30,                  Positive Integer\np=2,                           1 (Manhattan distance),   2(Elucidean distance)\nmetric=\'minkowski\',            "“euclidean”, “manhattan”, “chebyshev”, “minkowski”\n                               “wminkowski”, “seuclidean”, “mahalanobis”"\nmetric_params=None,            Dictionary\nn_jobs=None                    -1, 1 or -2\n)\n\n'

In [170]:
#Step1: Design the parameter grid. In Python, you can represent parameter grid in the form of Dictionary

weightsParameter = ['uniform','distance']
n_neighborsParameter = np.arange(1,31)
algorithmParameter = ['auto', 'ball_tree', 'kd_tree', 'brute']
metricParameter = ['euclidean', 'manhattan', 'chebyshev' , 'minkowski', 'seuclidean', 'mahalanobis']


paramGrid = dict(n_neighbors = n_neighborsParameter,
                weights= weightsParameter,
                algorithm = algorithmParameter,
                metric=metricParameter)

# Step2: Initialize the algo

from sklearn.neighbors import KNeighborsRegressor
modelGridSearch = KNeighborsRegressor()

#Step3: Search the best parameter for your data

#GridSearchCV


from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(modelGridSearch,
                   param_grid=paramGrid,
                   cv = 10) #Same as cross_val_score

In [171]:
# Step4: Extract Results

grid.fit(features,label)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['euclidean', 'manhattan', 'chebyshev',
                                    'minkowski', 'seuclidean', 'mahalanobis'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
                         'weights': ['uniform', 'distance']})

In [172]:
grid.best_score_

-9.180932748418233

In [173]:
grid.best_estimator_

KNeighborsRegressor(metric='euclidean')

In [174]:
grid.best_params_

{'algorithm': 'auto',
 'metric': 'euclidean',
 'n_neighbors': 5,
 'weights': 'uniform'}

In [175]:
finalModel = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [178]:
# 3. To extract the best training sample that gives the best score for KNeighborsRegressor

# Step1: Initialize the algo
#from sklearn.neighbors import KNeighborsRegressor
#modelTest = KNeighborsRegressor()

# Step2: Initialize K-Fold Cross Validation function

from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, #Use the same CV values that was applied in cross_val_score
             shuffle=True,
             random_state = 1) # To ensure the data is not randomized at every iteration

# 3. initialize for loop to identify which sample gives the best score and which sample is the best 
#.   training sample

counter = 0

for train,test in kfold.split(features):
    
    #Counter will help you track the sample split
    counter += 1
    
    #Extract the training set and testing set
    X_train,X_test = features[train],features[test]
    y_train,y_test = label[train] , label[test]
    
    #Fit the model 
    finalModel.fit(X_train,y_train)

    if finalModel.score(X_test,y_test) >= 0.8:
       print("Test Score {}, Train Score {} for Sample Split {}".format(finalModel.score(X_test,y_test),finalModel.score(X_train,y_train),counter))

Test Score 0.9698645019723984, Train Score 0.9490707851165177 for Sample Split 1
Test Score 0.9463413631002947, Train Score 0.939325042435145 for Sample Split 2
Test Score 0.8175128802662227, Train Score 0.9618423677474436 for Sample Split 4
Test Score 0.8802723288889419, Train Score 0.9548311766994682 for Sample Split 6
Test Score 0.8334705324417688, Train Score 0.9593927337275211 for Sample Split 7
Test Score 0.9783274092181723, Train Score 0.945124231648262 for Sample Split 8
Test Score 0.9597975025427307, Train Score 0.9549506955543277 for Sample Split 10


In [179]:
finalModel.score(X_test,y_test)

0.9597975025427307

In [53]:
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


In [180]:
# Extract the samples
# Step1: Initialize the algo
from sklearn.neighbors import KNeighborsRegressor
modelTest = KNeighborsRegressor()

# Step2: Initialize K-Fold Cross Validation function

from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, #Use the same CV values that was applied in cross_val_score
             shuffle=True,
             random_state = 1) # To ensure the data is not randomized at every iteration

# 3. initialize for loop to identify which sample gives the best score and which sample is the best 
#.   training sample

counter = 0
for train,test in kfold.split(features):
    
    #Counter will help you track the sample split
    counter += 1
    
    if counter == 1:
        X_train,X_test,y_train,y_test = features[train],features[test],label[train] , label[test]
    
 
    

In [181]:
kfold.split(features)

<generator object _BaseKFold.split at 0x29f00a500>

In [182]:
from sklearn.neighbors import KNeighborsRegressor
finalModel = KNeighborsRegressor()
finalModel.fit(X_train,y_train)
finalModel.score(X_test,y_test)

0.9698645019723984

In [ ]:
''' 
KNeighborsRegressor,
LinearRegression,
RandomForestRegressor, 
DecisionTreeRegressor all give negative values.
I'm trying svm and gradient boosting, if that doesn't work then boosting
'''

#result for minimum score from another student. (just have it to compare with mine)

#Algorithm: KNeighborsRegressor:  score.mean(): -9.180932748418233 
#Algorithm: LinearRegression; score.mean(): -5.36154501542167 
#Algorithm: Ridge; score.mean(): -5.321536240939532 
#Algorithm: Lasso; score.mean(): -5.359454343797066 
#Algorithm: SVR; score.mean(): -71.61028643587005 
#Algorithm: DecisionTreeRegressor; score.mean(): -13.275206556089353 
#Algorithm: RandomForestRegressor; score.mean(): -11.003588853560279

''' 
tried all the regressors I knew at cv=10 and got horrible results,
changed cvt to 5 and went back to the linear regressor and
found a favorable set of train/test data and got a final model test score at 0.985,
so I guess... if I've done it right,
I could say that the noted CL is achievable yet it takes very careful selection of splits with means bad
but there are a few good spots on the max side.
I say that with caution as the distribution of results is widely divergent.
'''

''' 
I'm running the XBGboostingRegressor with a grid search. Taking forever.
'''

''' 
#KNeighborsRegressor -13.43588015457872 
#LinearRegression -13.43588015457872 
#Random Forest Regressor 0.9943311099704247 0.9900294965255345 
#Decision Tree Regressor 0.9997560609325686 0.9986157840020002 
#XGBoost: (I used Regressor for XGBoost)
 Test : 0.8990283826136631 Train: 0.8831413505997752 RS: 1 
 Test : 0.8885681990121184 Train: 0.8873581704751071 RS: 2 
 Test : 0.9408557499271855 Train: 0.887162877264517 RS: 3 
 Test : 0.9130436650669594 Train: 0.8982483398878128 RS: 4 
 Test : 0.9066007611880108 Train: 0.8798352225364479 RS: 8 
 Test : 0.9135009638877835 Train: 0.8890726702949123 RS: 10 
 Test : 0.9211583984841318 Train: 0.8877585247195765 RS: 11 
 Test : 0.9265103822461054 Train: 0.8940650907921229 RS: 12 
 Test : 0.9246043552813843 Train: 0.8863996889153201 RS: 15 
 Test : 0.8992791509253291 Train: 0.8964649984406764 RS: 17 
 Test : 0.9585460293356046 Train: 0.8853890872909793 RS: 18 
 Test : 0.9238989935437033 Train: 0.8985493208721518 RS: 19 
 Test : 0.9182179346684616 Train: 0.8918856055807115 RS: 21 
 Test : 0.9552338091339604 Train: 0.8875759198
'''

''' 
markG -- what were the cues that LinearRegression was most promising model? (for me score.mean() was -0.33 for it)

answer:had a good max so I figured I'd go after the needle in the haystack so to speak...
I'm sure there are similar opportunities with other algorithms...
'''

'''  
if we decrease the CV to 2 or 3, lin regression gets quite good.
 At CV that low, does this turn the regression model into a de facto classifier?
'''

In [ ]:
''' 
if cross_val_score doesnt give you a good answer, the recommendation is use k-fold, shuffle spread or StratifiedKFold

'''